
<h1 align="center">Cell Tracking according to Biological Needs</h1>
<h2 align="center">Strong Mitosis-aware Random-finite Sets Tracker with Aleatoric Uncertainty</h2>

<p align="center">
  <img alt="Teaser Image" src="./assets/teaser.png">
  <br>
    <em>The output distribution of EmbedTrack using different input transformations on BF-C2DL-HSC.</em>
</p>

# Select the Dataset you want to process

In [1]:
# Select the dataset you want to process
DATASET = 'DIC-C2DH-HeLa'
SUBSET = 'train'
SEQUENCE = '01'

# Select some Inference Parameter
SHIFTS = 50
MULTISCALE = True
MULTI_SEGMENTATION = True

# Check if the selection is valid
AVAILABLE_DATASETS = [
    "Fluo-N2DH-SIM+",
    "Fluo-C2DL-MSC",
    "Fluo-N2DH-GOWT1",
    "PhC-C2DL-PSC",
    "BF-C2DL-HSC",
    "Fluo-N2DL-HeLa",
    "BF-C2DL-MuSC",
    "DIC-C2DH-HeLa",
    "PhC-C2DH-U373",
]
AVAILABLE_SUBSETS = ['train', 'challenge']
AVAILABLE_SEQUENCES = ['01', '02']
assert DATASET in AVAILABLE_DATASETS, f"Invalid dataset: {DATASET}"
assert SUBSET in AVAILABLE_SUBSETS, f"Invalid subset: {SUBSET}"
assert SEQUENCE in AVAILABLE_SEQUENCES, f"Invalid sequence: {SEQUENCE}"

# Setting up the environment

Creating the environment
  - cloning the code
  - downloading the CTC data
  - downloading the pretrained models. 
  
By default, everything will NOT be stored in your personal Google Drive folder 
(**/content/drive/MyDrive'**).
If you want to store the data in your drive, please uncomment the lines in the 
next cell of this notebook.

In [2]:
# The directory where the data will be stored
import os
prefix='colab_env'
!mkdir -p $prefix

# Uncomment this lines to store the data into your drive
#from google.colab import drive
#drive.mount('/content/drive')
#prefix='/content/drive'

os.chdir(prefix)

In [3]:
# Clear the directory
!ls
!rm -rf BiologicalNeeds

BiologicalNeeds


In [4]:
# Clone the repository
!git clone https://github.com/TimoK93/BiologicalNeeds
os.chdir('BiologicalNeeds')

Cloning into 'BiologicalNeeds'...
remote: Enumerating objects: 123, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 123 (delta 62), reused 114 (delta 53), pack-reused 0
Receiving objects: 100% (123/123), 255.56 KiB | 10.65 MiB/s, done.
Resolving deltas: 100% (62/62), done.


In [5]:
# Install the requirements
!conda env create -f environment.yml
!conda activate biological_needs


CondaValueError: prefix already exists: /home/timo/miniconda3/envs/biological_needs


CondaError: Run 'conda init' before 'conda activate'



In [6]:
# Download the data
if SUBSET == 'train':
    !python utils/prepare_data.py --datasets $DATASET --train
else:
    !python utils/prepare_data.py --datasets $DATASET --challenge

    http://data.celltrackingchallenge.net/training-datasets/DIC-C2DH-HeLa.zip /mnt/c/Users/kaiser/code/BiologicalNeeds/colab_env/BiologicalNeeds/Data/train
Unzip data set DIC-C2DH-HeLa.zip


In [9]:
# Download the pretrained models
import zipfile

!wget https://www.tnt.uni-hannover.de/de/project/MPT/data/BiologicalNeeds/models.zip
with zipfile.ZipFile("models.zip") as z:
    z.extractall("")

--2024-05-08 10:12:15--  https://www.tnt.uni-hannover.de/de/project/MPT/data/BiologicalNeeds/models.zip
Resolving www.tnt.uni-hannover.de (www.tnt.uni-hannover.de)... 130.75.31.71
Connecting to www.tnt.uni-hannover.de (www.tnt.uni-hannover.de)|130.75.31.71|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248308697 (237M) [application/zip]
Saving to: ‘models.zip.1’

models.zip.1        100%[===================>] 236.80M  31.8MB/s    in 7.6s    

2024-05-08 10:12:22 (31.2 MB/s) - ‘models.zip.1’ saved [248308697/248308697]

/bin/bash: unzip: command not found


# Inference

## 1) EmbedTrack with Aleatoric Uncertainty by Advanced Test-Time Augmentation

In [7]:
from embedtrack.scripts.inference import process

res_path = os.path.join(os.getcwd(), 'results/embedtrack')
process(
    train=True if SUBSET == 'train' else False,
    challenge=True if SUBSET == 'challenge' else False,
    dataset=DATASET,
    sequence=SEQUENCE,
    res_path=res_path,
    shifts=SHIFTS,
    multiscale=MULTISCALE,
    multi_segmentation=MULTI_SEGMENTATION,
)

## 2) Multi Hypothesis Tracking

In [ ]:
from mht.scripts.inference import infer_all

in_path = os.path.join(os.getcwd(), 'results/embedtrack')
res_path = os.path.join(os.getcwd(), 'results/mht')

infer_all(
    data_root=in_path,
    dest_root=res_path,
    subset=SUBSET,
    dataset=DATASET,
    sequence=SEQUENCE,
    tracker_args={},
)

## 3) Interpolation to fill temporal gaps in trajectories

In [ ]:
from utils.interpolate import postprocess_all

in_path = os.path.join(os.getcwd(), 'results/mht')
res_path = os.path.join(os.getcwd(), 'results/interpolated')

postprocess_all(
    data_root=in_path,
    dest_root=res_path,
    subset=SUBSET,
    dataset=DATASET,
    sequence=SEQUENCE,
)

## 4) Postprocessing to satisfy the CTC requirements

In [ ]:
from utils.postprocess import postprocess_all

res_path = os.path.join(os.getcwd(), 'results/interpolated')

postprocess_all(
    data_root=res_path,
    subset=SUBSET,
    dataset=DATASET,
    sequence=SEQUENCE,
)